# Make basic plots

## Importing packages

In [ ]:
from coffea import hist
from coffea.analysis_objects import JaggedCandidateArray
#import coffea.nanoevents.methods
import coffea.processor as processor
import awkward1 as ak
import numpy as np
import matplotlib.pyplot as plt
import os, sys, time, uproot_methods.classes.TVector3
from enum import Enum

## Global configuration

In [ ]:
# Add coffea packages to the import path
venv_path=os.environ['VIRTUAL_ENV']
site_path=venv_path+"/lib/python3.6/site-packages/"
sys.path.insert(0, site_path)

# Add local packages to the import path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# XRootD settings
xrootd_endpoint="root://hepxrd01.colorado.edu:1094/"
xrootd_base_path="/store/user/aperloff/ExoEMJAnalysis2020/"

# Dask settings
useDask = False
if useDask:
    from distributed import Client
    client = Client('coffea-dask.fnal.gov:8786')

# Print settings
verbose = False

# Debug settings
debug = True
if debug:
    verbose = True
    xrootd_base_path="/store/user/aperloff/ExoEMJAnalysis2020/Run2ProductionV18eDebug/"

## Get the sample data

In [ ]:
# The first and last (reload) import lines allow for making changes
#  in the xrootd module and reloading it in the same session.
import utils.python.xrootd
from utils.python.xrootd import *
importlib.reload(utils.python.xrootd)

condorSub_dicts = ["EMJ2016","EMJ2017","EMJ2018",
                   "Summer16v3_qcd","Fall17_qcd","Autumn18_qcd",
                   "Summer16v3_gjets","Fall17_gjets","Autumn18_gjets"]
sample_dict = get_files_xrootd(xrootd_endpoint,
                               xrootd_base_path,
                               dicts=condorSub_dicts,
                               verbose=verbose,
                               debug=((3,2) if debug else None)
                              )

## Setup the processor

In [ ]:
class EMJProcessor(processor.ProcessorABC):

    class Charge(Enum):
        FAILED = -1
        NEUTRAL = 0
        CHARGED = 1
    
    def __init__(self):
        dataset_axis = hist.Cat("dataset", "Primary dataset")
        pt_axis = hist.Bin("pt", r"$p_{T}$ [GeV]", 
                           np.array([0,5,10,15,20,25,30,35,40,45,50,60,70,80,90,
                                     100,120,140,160,180,
                                     200,250,300,350,400,450,500,
                                     600,700,800,900,1000,
                                     1500,2000,3000,4000,5000]))
        eta_axis = hist.Bin("eta", r"$\eta$", 20, -5, 5)
        phi_axis = hist.Bin("phi", r"$\phi$",64, -3.2, 3.2)
        e_axis = hist.Bin("e", r"energy$ [GeV]", 200, 0, 500)
        chi2_axis = hist.Bin("chi2",r"$\chi^{2}/NDF$", 20, 0, 20)
        ip2d_axis = hist.Bin("ip2d",r"IP_{2D}", 100, 0, 1)
        ip2d_sig_axis = hist.Bin("ip2d_sig",r"IP_{2D,sig}", 100, 0, 5)
        ipz_axis = hist.Bin("ipz",r"IP_{z}", 100, 0, 10)
        nhits_axis = hist.Bin("nhits",r"N_{hits}", 40, 0, 40)
        npix_axis = hist.Bin("npix",r"N_{pix}", 10, 0, 10)
        tknum_axis = hist.Bin("tknum", r"N_{track}", 50, 0, 500)
        partnum_axis = hist.Bin("partnum", r"N_{particles}", 500, 0, 5000)
        pdgid_axis = hist.Bin("pdgid", r"PDGID", 5000, 0, 5000)
        x_axis = hist.Bin("x", r"X", 600, -30, 30)
        y_axis = hist.Bin("y", r"Y", 600, -30, 30)
        z_axis = hist.Bin("z", r"Z", 600, -30, 30)
        rho_axis = hist.Bin("rho", r"\rho", 600, 0, 30)
        mult_axis = hist.Bin("mult", r"Multiplicity", 70, 0, 70)
        
        self._accumulator = processor.dict_accumulator({
            # Final state particle counting Particle counting
            'NumFinal' : hist.Hist("Counts", dataset_axis, partnum_axis),
            'NumChargeFinal' : hist.Hist("Counts", dataset_axis, partnum_axis),
            'FinalPDGID' : hist.Hist("Counts", dataset_axis, pdgid_axis),
            # Unlike reco::GenParticles used in https://gitlab.cern.ch/yichen/emj-analyze/-/blob/master/plugins/GenHistogram.cc
            #  we don't store the vertex position associated to the GenParticle
            #'FinalGenX' : hist.Hist("Counts", dataset_axis, x_axis),
            #'FinalGenY' : hist.Hist("Counts", dataset_axis, y_axis),
            #'FinalGenZ' : hist.Hist("Counts", dataset_axis, z_axis),
            'FinalPt' : hist.Hist("Counts", dataset_axis, pt_axis),
            'FinalEta' : hist.Hist("Counts", dataset_axis, eta_axis),
            'FinalPhi' : hist.Hist("Counts", dataset_axis, phi_axis),
            'FinalE' : hist.Hist("Counts", dataset_axis, e_axis),

            # Hidden valley particles from decay chain
            "DecaySummary" : hist.Hist("Counts", dataset_axis, mult_axis),
            "DecayDQuarkMult" : hist.Hist("Counts", dataset_axis, mult_axis),
            "DecayDMesonMult_Imm" : hist.Hist("Counts", dataset_axis, mult_axis),
            "DecayDMesonMult_All" : hist.Hist("Counts", dataset_axis, mult_axis),
            "DarkDecayGenRho" : hist.Hist("Counts", dataset_axis, rho_axis),
            "DarkDecayGenEta": hist.Hist("Counts", dataset_axis, eta_axis),
            "DarkDecayGenZ" : hist.Hist("Counts", dataset_axis, z_axis),
            
            # Reco track plots
            'tkpt' : hist.Hist("Counts", dataset_axis, pt_axis),
            'tketa' : hist.Hist("Counts", dataset_axis, eta_axis),
            'tkphi' : hist.Hist("Counts", dataset_axis, phi_axis),
            'tknum' : hist.Hist("Counts", dataset_axis, tknum_axis),
            'tkchi2' : hist.Hist("Counts", dataset_axis, chi2_axis),
            'tkip2d' : hist.Hist("Counts", dataset_axis, ip2d_axis),
            'tkip2d_sig' : hist.Hist("Counts", dataset_axis, ip2d_sig_axis),
            'tkipz' : hist.Hist("Counts", dataset_axis, ipz_axis),
            'tknhits' : hist.Hist("Counts", dataset_axis, nhits_axis),
            'tknpix' : hist.Hist("Counts", dataset_axis, npix_axis),

            # Reco jet plots
            'jtpt':hist.Hist("Counts", dataset_axis, pt_axis),
            'jteta':hist.Hist("Counts", dataset_axis, eta_axis),
            'jtphi':hist.Hist("Counts", dataset_axis, phi_axis),
            'jte':hist.Hist("Counts", dataset_axis, e_axis),
            
            # Simple accumulators
            'cutflow': processor.defaultdict_accumulator(int),
        })
    
    @property
    def accumulator(self):
        return self._accumulator
    
    def process(self, df):
        output = self.accumulator.identity()

        dataset = df['dataset']
        
        Jets = JaggedCandidateArray.candidatesfromcounts(
            df['Jets'],
            pt=df['Jets.fCoordinates.fPt'],
            eta=df['Jets.fCoordinates.fEta'],
            phi=df['Jets.fCoordinates.fPhi'],
            energy=df['Jets.fCoordinates.fE'],
            axismajor=df['Jets_axismajor'],
            axisminor=df['Jets_axisminor'],
            bDiscriminatorCSV=df['Jets_bDiscriminatorCSV'],
            bJetTagDeepCSVBvsAll=df['Jets_bJetTagDeepCSVBvsAll'],
            chargedEmEnergyFraction=df['Jets_chargedEmEnergyFraction'],
            chargedHadronEnergyFraction=df['Jets_chargedHadronEnergyFraction'],
            chargedHadronMultiplicity=df['Jets_chargedHadronMultiplicity'],
            chargedMultiplicity=df['Jets_chargedMultiplicity'],
            electronEnergyFraction=df['Jets_electronEnergyFraction'],
            electronMultiplicity=df['Jets_electronMultiplicity'],
            hadronFlavor=df['Jets_hadronFlavor'],
            hfEMEnergyFraction=df['Jets_hfEMEnergyFraction'],
            hfHadronEnergyFraction=df['Jets_hfHadronEnergyFraction'],
            HTMask=df['Jets_HTMask'],
            ID=df['Jets_ID'],
            jecFactor=df['Jets_jecFactor'],
            jecUnc=df['Jets_jecUnc'],
            jerFactor=df['Jets_jerFactor'],
            jerFactorDown=df['Jets_jerFactorDown'],
            jerFactorUp=df['Jets_jerFactorUp'],
            LeptonMask=df['Jets_LeptonMask'],
            MHTMask=df['Jets_MHTMask'],
            multiplicity=df['Jets_multiplicity'],
            muonEnergyFraction=df['Jets_muonEnergyFraction'],
            muonMultiplicity=df['Jets_muonMultiplicity'],
            neutralEmEnergyFraction=df['Jets_neutralEmEnergyFraction'],
            neutralHadronEnergyFraction=df['Jets_neutralHadronEnergyFraction'],
            neutralHadronMultiplicity=df['Jets_neutralHadronMultiplicity'],
            neutralMultiplicity=df['Jets_neutralMultiplicity'],
            origIndex=df['Jets_origIndex'],
            partonFlavor=df['Jets_partonFlavor'],
            photonEnergyFraction=df['Jets_photonEnergyFraction'],
            photonMultiplicity=df['Jets_photonMultiplicity'],
            ptD=df['Jets_ptD'],
            qgLikelihood=df['Jets_qgLikelihood'],
        )    
        
        GenJets = JaggedCandidateArray.candidatesfromcounts(
            df['GenJets'],
            pt=df['GenJets.fCoordinates.fPt'],
            eta=df['GenJets.fCoordinates.fEta'],
            phi=df['GenJets.fCoordinates.fPhi'],
            energy=df['GenJets.fCoordinates.fE'],
        )

        GenParticles = JaggedCandidateArray.candidatesfromcounts(
            df['GenParticles'],
            pt=df['GenParticles.fCoordinates.fPt'],
            eta=df['GenParticles.fCoordinates.fEta'],
            phi=df['GenParticles.fCoordinates.fPhi'],
            energy=df['GenParticles.fCoordinates.fE'],
            #Charge=df['GenParticles.Charge'],
            ParentId=df['GenParticles_ParentId'],
            ParentIdx=df['GenParticles_ParentIdx'],
            PdgId=df['GenParticles_PdgId'],
            Status=df['GenParticles_Status'],
        )

        PrimaryVertices = JaggedCandidateArray.candidatesfromcounts(
            df['PrimaryVertices_position.fCoordinates.fX'],
            px=df['PrimaryVertices_position.fCoordinates.fX'],
            py=df['PrimaryVertices_position.fCoordinates.fY'],
            pz=df['PrimaryVertices_position.fCoordinates.fZ'],
            mass=np.zeros_like(df['PrimaryVertices_position.fCoordinates.fX']),
            chi2=df['PrimaryVertices_chi2'],
            isFake=df['PrimaryVertices_isFake'],
            isGood=df['PrimaryVertices_isGood'],
            isValid=df['PrimaryVertices_isValid'],
            ndof=df['PrimaryVertices_ndof'],
            nTracks=df['PrimaryVertices_nTracks'],
            position=df['PrimaryVertices_position'],
            tError=df['PrimaryVertices_tError'],
            time=df['PrimaryVertices_time'],
            xError=df['PrimaryVertices_xError'],
            yError=df['PrimaryVertices_yError'],
            zError=df['PrimaryVertices_zError'],
        )
        
        #Access with Tracks.p4.{x,y,z,mass/energy=t}
        Tracks = JaggedCandidateArray.candidatesfromcounts(
            df['Tracks'],
            px=df['Tracks.fCoordinates.fX'],
            py=df['Tracks.fCoordinates.fY'],
            pz=df['Tracks.fCoordinates.fZ'],
            mass=np.zeros_like(df['Tracks.fCoordinates.fX']),
            charge=df['Tracks_charge'],
            dxyErrorPV0=df['Tracks_dxyErrorPV0'],
            dxyPV0=df['Tracks_dxyPV0'],
            dzAssociatedPV=df['Tracks_dzAssociatedPV'],
            dzErrorPV0=df['Tracks_dzErrorPV0'],
            dzPV0=df['Tracks_dzPV0'],
            etaError=df['Tracks_etaError'],
            firstHit=df['Tracks_firstHit'],
            foundHits=df['Tracks_foundHits'],
            fromPV0=df['Tracks_fromPV0'],
            hitPattern=df['Tracks_hitPattern'],
            hitPatternOffsets=df['Tracks_hitPatternOffsets'],
            IP2DPV0=df['Tracks_IP2DPV0'],
            IP2dSigPV0=df['Tracks_IP2dSigPV0'],
            IP3DPV0=df['Tracks_IP3DPV0'],
            IP3DSigPV0=df['Tracks_IP3DSigPV0'],
            IPzPV0 = np.sqrt(df['Tracks_IP3DPV0']**2 - df['Tracks_IP2DPV0']**2),
            lostHits=df['Tracks_lostHits'],
            matchedToPFCandidate=df['Tracks_matchedToPFCandidate'],
            normalizedChi2=df['Tracks_normalizedChi2'],
            numberOfHits=df['Tracks_numberOfHits'],
            numberOfPixelHits=df['Tracks_numberOfPixelHits'],
            phiError=df['Tracks_phiError'],
            ptError=df['Tracks_ptError'],
            pvAssociationQuality=df['Tracks_pvAssociationQuality'],
            qoverpError=df['Tracks_qoverpError'],
            quality=df['Tracks_quality'],
        )
        Tracks_referencePoint=ak.zip({"x": df['Tracks_referencePoint.fCoordinates.fX'],
                                      "y": df['Tracks_referencePoint.fCoordinates.fY'],
                                      "z": df['Tracks_referencePoint.fCoordinates.fZ']},
                                     with_name="ThreeVector")
        
        
        evtweights = df["Weight"].reshape(-1, 1).flatten()
        output['cutflow']['all events'] += Jets.size

        jetId_cut = (Jets.ID > 0)        
        Jets = Jets[jetId_cut]
        output['cutflow']['>=1 with loose id'] += jetId_cut.any().sum()
        twoJets = (Jets.counts >= 2)        
        output['cutflow']['>=2 reco jets'] += twoJets.sum()
        twoGens = (GenJets.counts >= 2)
        output['cutflow']['>=2 gen jets'] += twoGens.sum()
        
        #Jets = Jets[twoJets & twoGens]
        #GenJets = GenJets[twoJets & twoGens]
        
        
        #dphi_index = Jets.p4[:,0].delta_phi( Jets.p4[:,1] ) > 1.8
        #output['cutflow']['dPhi > 1.8'] += dphi_index.sum()
        

        #Jets = Jets[dphi_index]
        #GenJets = GenJets[dphi_index]
        
        #pairing = Jets.p4[:,0:2].cross(GenJets.p4, nested=True)
        #metric = pairing.i0.delta_r(pairing.i1)
        
        #index_of_minimized = metric.argmin()
        #dr_cut = (metric[index_of_minimized] < 0.2)
        #best_pairings_that_pass_dr_cut = pairing[index_of_minimized][dr_cut]
        #genrecos = best_pairings_that_pass_dr_cut.flatten(axis=1)
        #ptresponse = genrecos.i0.pt / genrecos.i1.pt
        
        #
        # Calculate non-trivial GEN quantities
        #
        
        # Mask and collection for only status==1 GenParticles
        genParticles_status1_mask = (GenParticles.Status == 1)
        genParticles_status1 = GenParticles[genParticles_status1_mask]
        
        # Mask and collection for the charged particles
        # Currently the charge isn't stored.
        #genParticles_charged_mask = (GenParticles.Charge != 0)
        # This is a temporary pass-through
        genParticles_charged_mask = (GenParticles.Status)
        
        # Calculate the combined filter and collection
        genParticles_status1_charged_mask = (genParticles_status1_mask & genParticles_charged_mask)
        num_final         = genParticles_status1_charged_mask.sum()
        num_final_charged = 0

        #
        # Fill the GEN histograms
        #
        output['FinalPDGID'].fill(dataset=dataset, pdgid=genParticles_status1.PdgId.flatten())
        output['FinalPt'].fill(dataset=dataset, pt=genParticles_status1.pt.flatten())
        output['FinalEta'].fill(dataset=dataset, eta=genParticles_status1.eta.flatten())
        output['FinalPhi'].fill(dataset=dataset, phi=genParticles_status1.phi.flatten())
        output['FinalE'].fill(dataset=dataset, e=genParticles_status1.p4.energy.flatten())
        
        #
        # Fill the RECO histograms
        #
        output['jtpt'].fill(dataset=dataset, pt=Jets.pt.flatten())
        output['jteta'].fill(dataset=dataset, eta=Jets.eta.flatten())
        output['jtphi'].fill(dataset=dataset, phi=Jets.phi.flatten())
        output['jte'].fill(dataset=dataset, e=Jets.p4.energy.flatten())
        output['tkpt'].fill(dataset=dataset, pt=Tracks.pt.flatten())
        output['tketa'].fill(dataset=dataset, eta=Tracks.eta.flatten())
        output['tkphi'].fill(dataset=dataset, phi=Tracks.phi.flatten())
        output['tknum'].fill(dataset=dataset, tknum=Tracks.counts)
        output['tkchi2'].fill(dataset=dataset, chi2=Tracks.normalizedChi2.flatten())
        output['tkip2d'].fill(dataset=dataset, ip2d=Tracks.IP2DPV0.flatten())
        output['tkip2d_sig'].fill(dataset=dataset, ip2d_sig=Tracks.IP2dSigPV0.flatten())
        output['tkipz'].fill(dataset=dataset, ipz=Tracks.IPzPV0.flatten())
        output['tknhits'].fill(dataset=dataset, nhits=Tracks.numberOfHits.flatten())
        output['tknpix'].fill(dataset=dataset, npix=Tracks.numberOfPixelHits.flatten())

        return output

    def postprocess(self, accumulator):
        return accumulator

In [ ]:
tstart = time.time() 

if not useDask:
    output = processor.run_uproot_job(sample_dict,
                                      treename='TreeMaker2/PreSelection',
                                      processor_instance=EMJProcessor(),
                                      executor=processor.iterative_executor,
                                      executor_args={
                                          'skipbadfiles':True,
                                          'nano':False, 
                                          'flatten':True, 
                                          'workers': 4},
                                      chunksize=50000, maxchunks=100
                                     )
else:
    output = processor.run_uproot_job(sample_dict,
                                      treename='TreeMaker2/PreSelection',
                                      processor_instance=EMJProcessor(),
                                      executor=processor.dask_executor,
                                      executor_args={
                                          'skipbadfiles':True,
                                          'client': client, 
                                          'nano':False, 
                                          'flatten':True, 
                                          'workers': 2},
                                      chunksize=50000
                                     )

elapsed = time.time() - tstart
print(output)

In [ ]:
print("Events/s:", output['cutflow']['all events']/elapsed)

In [ ]:
# Everything contained in the 'output' dictionary
output

In [ ]:
# The cutflow accumulator
output['cutflow']

In [ ]:
# List all of the processes on the 'dataset' axis for the jtpt histogram
output['jtpt'].axis('dataset').identifiers()

In [ ]:
# Show the values in one of the histograms
output['jtpt']['EMJ_2016_mMed-1000_mDark-20_kappa-0p12_aligned-down'].values()

In [ ]:
from utils.python.accumulator import *
if debug:
    test_filter_histograms(output)
    print('\n\n')
    test_integrate_hist_over_dataset(output)

## Make Plots

### Configuration

In [ ]:
plt.rcParams["image.cmap"] = 'Blues'

### Jet Plots

In [ ]:
hnames = ['jtpt','jteta','jtphi','jte']
logx = [1,0,0,1]
logy = [1,0,0,1]
for ih, hname in enumerate(hnames):
    ax = hist.plotgrid(output[hname], overlay="dataset", stack=False, density=True,
                       #fill_opts=stack_fill_opts,
                       #error_opts=stack_error_opts,
                      )
    if logx[ih]: plt.xscale("log")
    if logy[ih]: plt.yscale("log")
    for iax in ax.flatten():
        iax.autoscale(axis='y')
    plt.show()

### Track Plots

In [ ]:
hnames = ['tkpt','tketa','tkphi','tknum','tkchi2','tkip2d','tkip2d_sig','tkipz','tknhits','tknpix']
logx = [1,0,0,0,0,0,0,0,0,0]
logy = [1,0,0,0,1,1,0,1,0,0]
for ih, hname in enumerate(hnames):
    ax = hist.plotgrid(output[hname], overlay="dataset", stack=False, density=True,
                       #fill_opts=stack_fill_opts,
                       #error_opts=stack_error_opts,
                      )
    if logx[ih]: plt.xscale("log")
    if logy[ih]: plt.yscale("log")
    for iax in ax.flatten():
        iax.autoscale(axis='y')
    plt.show()